# Topologias RNN

As APIs para arquiteturas MLP e CNN são limitadas. Ambas arquiteturas aceitam um tensor de dimensão fixa e produzem um tensor fixo de saída, realizam uma transformação desde a entrada até a saída em um número fixo de passos que é dada pelo número de camadas do modelo. RNNs não tem essa limitação. Você pode ter seqüências na entrada, a saída, ou ambas. Então, RNNs pode ser estruturada de várias formas para resolver algum problema específico.

As RNNs combinam um vetor de entrada com o vetor de estado anterior para produzir um novo vetor de estado. Isto pode ser pensado similar à rodar um programa com algumas entradas e algumas variáveis internas. Assim, RNNs pode pensado como essencialmente descrever um programa de computador.

De fato, foi provado que RNNs são sistemas **Turing-completo** (para mais informação ler o artigo: [One Computational Power of Neural Nets](http://ac.els-cdn.com/S0022000085710136/1-s2.0-S0022000085710136-main.pdf?_tid=248652fe-4305-11e7-b5c2-00000aab0f27&acdnat=1495907682_fabe262a44ec6ae2bf04df3bb2e8b88d), H.T. Siegelmann e E.D. Sontag) em essência dada a propriedade dos pesos, eles podem simular programas arbitrários.

Estas propriedades de ser capaz de trabalhar com seqüências da origem a um grande número de topologias, algumas de elas são apresentadas: 

![rnntypes](img/rnntypes.jpg)

Todas estas diferentes topologias da estrutura básica apresentada no tutorial anterior. Nesta básica topologia, todas as seqüências de entradas são do mesmo cumprimento e uma saída é produzida em cada intervalo de tempo. 

Um exemplo de **many to many RNN** pode ser uma maquina de tradução **(b)** parte de uma família geral chamada sequence-to-sequence (Para mais informação: [Grammar as a Foreign Language](https://arxiv.org/pdf/1412.7449.pdf), by Vinyals, Advances in Neural Information Processing Systems, 2015). Estas pegam uma seqüência e geram outra seqüência, no caso da máquina de tradução, a entrada pode ser uma seqüência de palavras em inglês numa frase e a saída pode ser as palavras numa frase em português.

Outra variante é a **one-to-many RNN**: Image Captioning network (para mais informação ler o artigo: ([Deep Visual-Semantic Alignments for Generating Image Descriptions](https://cs.stanford.edu/people/karpathy/cvpr2015.pdf), A. Karpathy, e F. Li, Proceeding of the IEEE Conference on Computer Vision and Pattern Recognition, 2015), onde a entrada é uma imagem e a saída, uma seqüência de palavras.

Um exemplo de **many-to-one RNN**: Análise sentimental de frases, onde a entrada é uma seqüência de palavras e a saída é um sentimento positivo ou negativo. (Para maior informação ler o artigo: [Recursive Deep Models for Semantic Compositionality over a Sentiment Treebank](https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf) R. Socher, Conference on Empirical Methods in Natural Language Processing, 2013 ), Apresentaremos um exemplo (simplificada em comparação ao modelo citado) desta topologia neste laboratório.

# Vanishing and exploding gradients

Assim como as redes neurais tradicionais, treinar uma RNN também envolve backpropagation. A diferencia neste caso é que como os parâmetros são compartilhados em todos os passos de tempo, a gradiente em cada saída depende não só do passo atual, também dos passos anteriores. Este processo é chamado Backpropagation através do tempo **(Backpropagation through time - BPTT)**, para mais informação ler o artigo: [Learning Internal Representations by Backpropagation errors](https://www.iro.umontreal.ca/~vincentp/ift3395/lectures/backprop_old.pdf), por G.E. Hinton, D.E. Rumelhart, e R.J. Williams, Parallel Distributed Processing: Explorations in the Microstructure of Cognition 1, 1985):

![](img/bptt.png)

Considere uma RNN de 3 camadas mostrada no gráfico. Durante o *forward propagation* (mostradas nas linhas solidas), a rede produze predições que são comparadas com os rótulos para calcular a função de perda em cada time step. Durante o *backpropagation* (mostrada pelos traços) a gradiente da função de perda com respeito aos parâmetros U, V e W são computadas em cada intervalo de tempo e os parâmetros são atualizados com a soma dos gradientes.

A seguinte equação mostra a gradiente da função de perda com respeito a W, a matriz que codifica os pesos para as dependências dos termos ao longo prazo.

\begin{equation}
\frac{\partial L}{\partial W} = \sum_{t}{\frac{\partial L}{\partial W}}
\end{equation}

Observemos agora o que acontece à gradiente da função de perda no último time-step (t=3). Como podem observar, o gradiente pode ser descomposto como o produto de 3 sub-gradientes utilizando a regra da cadeia. O gradiente do estado oculto h2 com respeito de W pode ser descompactado como a soma do gradiente de cada estado oculto com respeito do anterior. Finalmente, cada gradiente do estado oculto com respeito do estado anterior pode ser descomposto como o produto de gradientes de o estado atual oculto respeito o anterior.

\begin{eqnarray}
\frac{\partial L}{\partial W} &=& \frac{\partial L_{3}}{\partial y_{3}} . \frac{\partial y_{3}}{\partial h_{2}} . \frac{\partial h_{2}}{\partial W}\\
&=& \sum_{t=0}^{2}\frac{\partial L_{3}}{\partial y_{3}} . \frac{\partial y_{3}}{\partial h_{2}} . \frac{\partial h_{2}}{\partial h_{t}}.\frac{\partial h_{t}}{\partial W} \\
&=& \sum_{t=0}^{2}\frac{\partial L_{3}}{\partial y_{3}} . \frac{\partial y_{3}}{\partial h_{2}} . 
(\prod_{j=t+1}^{2} \frac{\partial h_{j}}{\partial h_{j-1}}) . \frac{\partial h_{t}}{\partial W}
\end{eqnarray}

Cálculos similares são realizados para calcular a gradiente da função de perda de $L_{1}$ e $L_{2}$ (no tempo 1 e 2) com respeito a W e somar-lo no gradiente atualizada de W. Se deseja profundar neste tema este [link](http://www.wildml.com/2015/10/recurrent-neural-networks-tutorial-part-3-backpropagation-through-time-and-vanishing-gradients/) tem uma boa explicação de BPTT, com maiores detalhes da derivação e a matemática do processo.

Para nosso propósito, a fórmula final das equações acima mostra porque RNN tem o problema de **vanishing and exploding gradients**. Considere o caso onde uma gradiente individual do estado oculto com respeito ao estado anteriores menor que 1. Como propagamos através de multiplex passos, o produto dos gradientes começa a ficar menor e menor, apresentando o problema de desaparecimento do gradiente (**vanishing**).

Similarmente, se a gradiente é maior que 1, a gradiente é maior e maior, apresentando o problema de exploração da gradiente (**exploding gradients**).

O effeito de:
* **Vanishing Gradients**: Os passos que estão longe de um determinado estado, não ajudam ao processo de aprendizagem. Assim, a RNN não consegue aprender longas dependências. O desaparecimento do gradiente também pode acontecer nas redes neurais tradicionais, e é mais visível que em RNNs, acontecendo quando a rede tem muitas camadas e backpropagation tem que passar por elas.

* **Exploding gradients**: O valor do gradiente começa a ficar tão grande que vira um NaN (Not a Number) é o processo de aprendizagem falha.

Embora existam poucas enfoque para minimizar estes problema. A solução mais popular é utilizar arquiteturas LSTM ou GRU. Estas arquiteturas foram desenhadas para lidar com estes tipos de problemas.